# Simple Agent System

This notebook demonstrates how to create a simple agent system using the LlamaIndex framework. This example uses a function-calling agent that can use tools to search and retrieve information.

## Setup and Installation

First, let's install the required packages and verify the installation:

In [ ]:
# Install required packages
!pip install llama-index llama-index-llms-openai nest-asyncio python-dotenv

# Verify installations
import importlib

def check_package(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False

packages = {
    "llama_index": "LlamaIndex Core",
    "llama_index.llms.openai": "OpenAI integration",
    "nest_asyncio": "Nest AsyncIO",
    "dotenv": "Python-dotenv"
}

all_installed = True
for package, display_name in packages.items():
    installed = check_package(package)
    print(f"{display_name}: {'✅ Installed' if installed else '❌ Not installed'}")
    all_installed = all_installed and installed

if all_installed:
    print("\n✅ All required packages are installed!")
else:
    print("\n⚠️ Some packages are missing. Run the installation command again.")

## Environment Setup

Load environment variables from the `.env` file. <br>
N.b. it will look through the entire project for a valid `.env` file.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Get model name from environment or use default
MODEL_NAME = os.getenv("SIMPLE_AGENT_MODEL", "gpt-4o-mini")

# Set environment variables for compatibility with libraries that expect them
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY or ""

# Verify API key is set
if not OPENAI_API_KEY:
    raise ValueError("No OpenAI API key found. Please set OPENAI_API_KEY in your .env file.")
else:
    print("✅ API key loaded successfully")
    print(f"✅ Using model: {MODEL_NAME}")

## Import Required Libraries

Let's import the necessary libraries for our agent system.

In [ ]:
import asyncio
import json
import logging
import sys
from IPython.display import Markdown, display

# Configure basic logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Configure LLM

Set up the OpenAI language model to be used by our agent.

In [ ]:
from llama_index.llms.openai import OpenAI

# Initialize the language model with API key from environment
llm = OpenAI(model=MODEL_NAME, api_key=OPENAI_API_KEY)

# Verify LLM setup with a simple test
response = llm.complete("Hello, I am a language model. ")
print("LLM Test Response:", response.text)

## Define Tools

Let's define some tools that our agent can use to perform tasks. We'll create two simple tools: a calculator and a web search simulator.

In [ ]:
from llama_index.core.tools import FunctionTool

# A simple calculator tool
async def calculator(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        # Safely evaluate the expression
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error evaluating {expression}: {str(e)}"

# A simulated web search tool
async def web_search(query: str) -> str:
    """Simulate searching the web for information."""
    # This is just a simulation - in a real application, you would use an actual search API
    search_results = {
        "python programming": "Python is a high-level, interpreted programming language known for its readability and versatility.",
        "machine learning": "Machine learning is a field of AI that enables computers to learn from data without being explicitly programmed.",
        "climate change": "Climate change refers to long-term shifts in temperatures and weather patterns, mainly caused by human activities."
    }
    
    # Find the most relevant key in our mock database
    for key in search_results:
        if key.lower() in query.lower():
            return search_results[key]
    
    return "No relevant information found for your query."

# Convert functions to LlamaIndex tools
calculator_tool = FunctionTool.from_defaults(fn=calculator)
search_tool = FunctionTool.from_defaults(fn=web_search)

print("✅ Tools defined successfully")

## Create the Agent

Now let's create a function-calling agent that can use our tools.

In [ ]:
from llama_index.core.agent import FunctionCallingAgent

# Create the agent with our tools
agent = FunctionCallingAgent.from_tools(
    tools=[calculator_tool, search_tool],
    llm=llm,
    verbose=True,  # Set to True to see the agent's thought process
    system_prompt="You are a helpful AI assistant that can perform calculations and search for information."
)

print("✅ Agent created successfully")

## Test the Agent

Let's try our agent with different types of queries.

In [ ]:
# Apply nest_asyncio to enable asynchronous code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Test the agent with a calculation query
print("Testing agent with calculation query...")
response = agent.chat("What is the result of 15 * 28 + 42?")
print("\nAgent Response:")
display(Markdown(f"**Answer:**\n\n{response}"))

In [ ]:
# Test the agent with an information query
print("Testing agent with information query...")
response = agent.chat("Tell me about Python programming.")
print("\nAgent Response:")
display(Markdown(f"**Answer:**\n\n{response}"))

In [ ]:
# Test the agent with a more complex query
print("Testing agent with complex query...")
response = agent.chat("If I have 5 apples and each costs $1.25, how much will I pay in total? Also, can you tell me about machine learning?")
print("\nAgent Response:")
display(Markdown(f"**Answer:**\n\n{response}"))

## Adding Memory to the Agent

Let's enhance our agent by adding memory so it can remember previous interactions.

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer

# Create a memory buffer
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

# Create a new agent with memory
agent_with_memory = FunctionCallingAgent.from_tools(
    tools=[calculator_tool, search_tool],
    llm=llm,
    memory=memory,
    verbose=True,
    system_prompt="You are a helpful AI assistant that can perform calculations and search for information. You maintain context from previous interactions."
)

print("✅ Agent with memory created successfully")

In [ ]:
# Test memory with a series of interactions
print("Testing agent memory with first interaction...")
response = agent_with_memory.chat("My name is Alice and I'm learning about climate change.")
print("\nAgent Response:")
display(Markdown(f"**Answer:**\n\n{response}"))

In [ ]:
# Test if the agent remembers the name
print("Testing agent memory with follow-up question...")
response = agent_with_memory.chat("What's my name and what am I learning about?")
print("\nAgent Response:")
display(Markdown(f"**Answer:**\n\n{response}"))

## Conclusion

In this notebook, we've demonstrated how to:
1. Set up a function-calling agent with LlamaIndex
2. Define custom tools for the agent to use
3. Test the agent with different types of queries
4. Enhance the agent with memory capabilities

This simple agent system can be expanded upon to include more sophisticated tools, such as API access, database queries, or integration with other systems.

## Next Steps

You can extend this example by:
1. Adding more complex tools
2. Integrating with real web search APIs
3. Implementing a user interface
4. Connecting to databases or other data sources

For a more advanced example, check out the [Multi Agent Collaboration](multi_agent_collaboration.ipynb) notebook.